In [1]:
import aocd
from aocd.models import Puzzle
day = 18
year = 2021
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [434]:
import functools
import json

In [378]:
data = puzzle.input_data.splitlines()
len(data), data[:10]

(100,
 ['[9,[[8,3],[6,9]]]',
  '[[[0,[5,0]],[9,[1,0]]],[[8,0],[6,[3,3]]]]',
  '[[5,[[4,1],[3,3]]],[[7,5],[[1,5],9]]]',
  '[8,3]',
  '[[[[0,5],3],2],[2,[6,0]]]',
  '[[[0,8],[7,5]],6]',
  '[[8,[9,[7,6]]],2]',
  '[[[[2,3],9],[0,0]],[8,[[8,2],6]]]',
  '[[[[8,7],[4,9]],[[0,1],9]],[[[2,1],[9,5]],2]]',
  '[[5,[6,0]],[[1,[6,5]],[[3,4],2]]]'])

In [310]:
test_data = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]""".splitlines()

In [428]:
def level(pair, l=1):
    for n in pair:
        if isinstance(n, list):
            print(n, l)
            level(n, l=l+1)
#         return pair, 'level={}'.format(l)

class Leaf():
    def __init__(self, value=None, parent=None):
        self.value = value
        self.parent = parent
        self.left = None
        self.right = None

    
    def walk(self):
        if isinstance(self.left, Leaf):
            self.left.walk()

        if isinstance(self.right, Leaf):
            self.right.walk()
        if self.value is not None:
            print(self.value)

            
    def find_left(self):
        this = self 
        try:
            while this.parent.left is this:
                this = this.parent
        except AttributeError:
            return None        
        this = this.parent.left        
#         print(this)
        if this is None:
            print('!!!')
        while this.value is None:
            this = this.right        
        return this
    
    def find_right(self):
        this = self 
        try:
            while this.parent.right is this:
                this = this.parent
        except AttributeError:
            return None        
        this = this.parent.right        
#         print(this)
        while this.value is None:
            this = this.left        
        return this
    
    def explode(self):
        left = self.find_left()
        right = self.find_right()
        
#         print("left: ", left)
#         print("right: ", right)
        
        if left is not None:
            left.value += self.left.value
        if right is not None:
            right.value += self.right.value
        
        self.value = 0
        del(self.left)
        del(self.right)
        self.left = None
        self.right = None
        
    def split(self):
        left = self.value // 2
        right = self.value // 2 + (self.value % 2)
        
        self.value = None
        self.left = Leaf(left)
        self.right = Leaf(right)
        self.left.parent = self
        self.right.parent = self
    
    def find_level(self, clvl=0, flvl=4):
        if clvl == flvl:
            if self.value is None:
                return self
            return None
        
        try:
        
            res_left = self.left.find_level(clvl=clvl+1, flvl=flvl)

            if res_left is not None:
                return res_left

            return self.right.find_level(clvl=clvl+1, flvl=flvl)
        except AttributeError:
            return None
    
    def find_value(self, thresh=10):
        if self.value is not None:
            if self.value >= thresh:
                return self
            return None
        
        res_left = self.left.find_value(thresh=thresh)
        if res_left is not None:
            return res_left
        
        return self.right.find_value(thresh=thresh)
    
    def magnitude(self):
        if self.value is not None:
            return self.value
        
        return 3 * self.left.magnitude() + 2 * self.right.magnitude()
        
    
    def __repr__(self):
        if self.value is not None:
            return str(self.value)
        return "[{}, {}]".format(self.left.__repr__(), self.right.__repr__())
    
    def __add__(self, other):
        new = Leaf()
        new.left = self
        new.right = other
        new.left.parent = new
        new.right.parent = new
        return new        
        

class SFnumber():
    def __init__(self, pairs=None):
        if pairs is not None:
            self.root = self.build(pairs)
        
    def build(self, pairs):
        leaf = Leaf()
        left, right = pairs
        if isinstance(left, list):
            leaf.left = self.build(left)
        else:
            leaf.left = Leaf(value=left)
        leaf.left.parent = leaf
            
        if isinstance(right, list):
            leaf.right = self.build(right)
        else:
            leaf.right = Leaf(value=right)
        leaf.right.parent = leaf
            
        return leaf    
            
    def reduce(self):
        while True:
            to_explode = self.root.find_level()
            if to_explode is not None:
#                 print("explode: ", to_explode)
                to_explode.explode()
#                 print(self)
                continue
            
            to_split = self.root.find_value()
            if to_split is None:
                break
#             print("split: ", to_split)
            to_split.split()
#             print(self)
            
    def __repr__(self):
        return self.root.__repr__()
            
    def __add__(self, other):
        new = SFnumber()
        new.root = self.root + other.root
        new.reduce()
        return new
    
        
            

In [425]:
num = SFnumber([[[[[9,8],1],2],3],4])
# print(num.root.left.left.left.left)
# num.root.left.left.left.left.explode()
print(num.root)

[[[[[9, 8], 1], 2], 3], 4]


In [286]:
num.reduce()
num

left:  None
right:  1


[[[[0, 9], 2], 3], 4]

In [287]:
num = SFnumber([7,[6,[5,[4,[3,2]]]]])
num.reduce()
print(num.root)

left:  4
right:  None
[7, [6, [5, [7, 0]]]]


In [298]:
num = SFnumber([[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]])
print(num.root.left.right.right.right)
num.root.left.right.right.right.explode()
print(num.root)

[7, 3]
left:  1
right:  6
[[3, [2, [8, 0]]], [9, [5, [4, [3, 2]]]]]


In [429]:
# Test
SFnumber([[[[4,3],4],4],[7,[[8,4],9]]]) + SFnumber([1,1])

[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]

In [402]:
# Test
SFnumber([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]).root.magnitude()

3488

In [403]:
test_data2="""[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]""".splitlines()

In [435]:
# numbers = list(map(json.loads, data))
numbers = list(map(json.loads, test_data2))

In [436]:
SFnum = SFnumber(numbers[0])
SFnum.reduce()
print(SFnum)
i=0
for num in numbers[1:]:
    print(i)
    add = SFnumber(num)
    print ("+ ", add)
    SFnum = SFnum + add
    print("= ", SFnum)

[[[0, [4, 5]], [0, 0]], [[[4, 5], [2, 6]], [9, 5]]]
0
+  [7, [[[3, 7], [4, 3]], [[6, 3], [8, 8]]]]
=  [[[[4, 0], [5, 4]], [[7, 7], [6, 0]]], [[8, [7, 7]], [[7, 9], [5, 0]]]]
0
+  [[2, [[0, 8], [3, 4]]], [[[6, 7], 1], [7, [1, 6]]]]
=  [[[[6, 7], [6, 7]], [[7, 7], [0, 7]]], [[[8, 7], [7, 7]], [[8, 8], [8, 0]]]]
0
+  [[[[2, 4], 7], [6, [0, 5]]], [[[6, 8], [2, 8]], [[2, 1], [4, 5]]]]
=  [[[[7, 0], [7, 7]], [[7, 7], [7, 8]]], [[[7, 7], [8, 8]], [[7, 7], [8, 7]]]]
0
+  [7, [5, [[3, 8], [1, 4]]]]
=  [[[[7, 7], [7, 8]], [[9, 5], [8, 7]]], [[[6, 8], [0, 8]], [[9, 9], [9, 0]]]]
0
+  [[2, [2, 2]], [8, [8, 1]]]
=  [[[[6, 6], [6, 6]], [[6, 0], [6, 7]]], [[[7, 7], [8, 9]], [8, [8, 1]]]]
0
+  [2, 9]
=  [[[[6, 6], [7, 7]], [[0, 7], [7, 7]]], [[[5, 5], [5, 6]], 9]]
0
+  [1, [[[9, 3], 9], [[9, 0], [0, 7]]]]
=  [[[[7, 8], [6, 7]], [[6, 8], [0, 8]]], [[[7, 7], [5, 0]], [[5, 5], [5, 6]]]]
0
+  [[[5, [7, 4]], 7], 1]
=  [[[[7, 7], [7, 7]], [[8, 7], [8, 7]]], [[[7, 0], [7, 7]], 9]]
0
+  [[[[4, 2], 2], 6], [8,

In [437]:
numbers = list(map(lambda x: SFnumber(json.loads(x)), test_data2))
total = functools.reduce(lambda a, b: a+b, numbers)
total.root.magnitude()

3488

In [438]:
numbers = list(map(lambda x: SFnumber(json.loads(x)), test_data))
total = functools.reduce(lambda a, b: a+b, numbers)
total.root.magnitude()

4140

In [439]:
numbers = list(map(lambda x: SFnumber(json.loads(x)), data))
total = functools.reduce(lambda a, b: a+b, numbers)
total, total.root.magnitude()

([[[[6, 6], [7, 7]], [[7, 7], [7, 7]]], [[[7, 7], [7, 0]], [[7, 8], [8, 7]]]],
 4120)

In [381]:
puzzle.answer_a = total.root.magnitude()

That's the right answer!  You are one gold star closer to finding the sleigh keys. [Continue to Part Two]


In [382]:
# Part B

In [440]:
%%time
numbers = list(map(json.loads, test_data))
num_numbers = len(numbers)
max_magnitude = 0
for i in range(num_numbers):
    for j in range(num_numbers):
        if i == j:
            continue
#         print(numbers[i], numbers[j])
        mag = (SFnumber(numbers[i]) + SFnumber(numbers[j])).root.magnitude()
        if mag > max_magnitude:
            max_magnitude = mag
print(max_magnitude)


3993
CPU times: user 64 ms, sys: 3.93 ms, total: 67.9 ms
Wall time: 66.1 ms


In [441]:
%%time
numbers = list(map(json.loads, data))
num_numbers = len(numbers)
max_magnitude = 0
for i in range(num_numbers):
    for j in range(num_numbers):
        if i == j:
            continue
#         print(numbers[i], numbers[j])
        mag = (SFnumber(numbers[i]) + SFnumber(numbers[j])).root.magnitude()
        if mag > max_magnitude:
            max_magnitude = mag
print(max_magnitude)


4725
CPU times: user 4.22 s, sys: 1.72 ms, total: 4.22 s
Wall time: 4.22 s


In [432]:
puzzle.answer_b = max_magnitude

That's the right answer!  You are one gold star closer to finding the sleigh keys.You have completed Day 18! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
